In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 150

In [13]:
def clean_countries_and_codes(df):
    df['clean_co_codes'] = df.loc[df['country_codes'].notnull(), 'country_codes'].str.replace(';XXX|XXX;', '')
    df['clean_co_codes'] = df.loc[df['clean_co_codes'].notnull(), 'clean_co_codes'].replace('XXX', np.nan)
    df['clean_co_codes'] = df.loc[df['clean_co_codes'].notnull(), 'clean_co_codes'].str.split(';').apply(lambda x: sorted(x))
    df['clean_co_codes'] = df.loc[df['clean_co_codes'].notnull(), 'clean_co_codes'].apply(', '.join)
    
    df['clean_countries'] = df.loc[df['countries'].notnull(), 'countries'].str.replace(';Not identified|Not identified;', '')
    df['clean_countries'] = df.loc[df['clean_countries'].notnull(), 'clean_countries'].replace('Not identified', np.nan)
    df['clean_countries'] = df.loc[df['clean_countries'].notnull(), 'clean_countries'].str.split(';').apply(lambda x: sorted(x))
    df['clean_countries'] = df.loc[df['clean_countries'].notnull(), 'clean_countries'].apply(' ,'.join)
    
    df.drop(columns=['country_codes', 'countries'], inplace=True)
    df.rename(columns={'clean_co_codes':'country_codes', 'clean_countries':'countries'}, inplace=True)

In [4]:
keep_cols = ['node_id', 'sourceID', 'address', 'name', 'country_codes', 'countries']
dtypes = {'node_id': 'int32', 'sourceID':'category', 'address': 'object', 'name':'object',
          'country_codes':'category', 'countries':'category'}

bahamas_address_raw = pd.read_csv('../data/raw/bahamas_leaks/bahamas_leaks.nodes.address.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
bahamas_address_raw = bahamas_address_raw[keep_cols]

offshore_address_raw = pd.read_csv('../data/raw/offshore_leaks/offshore_leaks.nodes.address.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
offshore_address_raw = offshore_address_raw[keep_cols]

panama_address_raw = pd.read_csv('../data/raw/panama_papers/panama_papers.nodes.address.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
panama_address_raw = panama_address_raw[keep_cols]

paradise_address_raw = pd.read_csv('../data/raw/paradise_papers/paradise_papers.nodes.address.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
paradise_address_raw = paradise_address_raw[keep_cols]

### To Do

- Paradise papers: address was accidentally put into the 'name' column instead of the 'address' column. Will need to rename this column.
  + However, Line 1 of the address is mostly in the 'address' column. Which could make reformatting that dataset easier.
- Remove all rows where the 'address' column is empty (there's only one each in Panama and Paradise (once 'name' column correctly addressed)
- Clean the address text. This will require rather inventive text wrangling. Each investigation uses a different address format. Not all have the City. There are invalid characters in some of the entries.
- Preferred address format (in different columns)
  + Line 1
  + City
  + State
  + Zip/postal code
  + Country
  + If you standardize and validate, also grab the lat/long

In [13]:
bahamas_address_raw.head(25)

# May be able to just split on ','

,node_id,sourceID,address,name,country_codes,countries
0,24000001,Bahamas Leaks,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS",NaN,BHS,Bahamas
1,24000002,Bahamas Leaks,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-8188, NASSAU, BAHAMAS",NaN,BHS,Bahamas
2,24000003,Bahamas Leaks,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS",NaN,BHS,Bahamas
3,24000004,Bahamas Leaks,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS",NaN,BHS,Bahamas
4,24000005,Bahamas Leaks,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS",NaN,BHS,Bahamas
5,24000006,Bahamas Leaks,"303 SHIRLEY STREET, P.O. BOX N-492, NASSAU, BAHAMAS",NaN,BHS,Bahamas
6,24000007,Bahamas Leaks,"OCEAN CENTRE, MONTAGU FORESHORE, P.O. BOX SS-19084 EAST BAY STREET, NASSAU, BAHAMAS",NaN,BHS,Bahamas
7,24000008,Bahamas Leaks,"PROVIDENCE HOUSE, EAST WING EAST HILL ST, P.O. BOX CB-12399, NASSAU, BAHAMAS",NaN,BHS,Bahamas
8,24000009,Bahamas Leaks,"BAYSIDE EXECUTIVE PARK, WEST BAY & BLAKE, P.O. BOX N-4875, NASSAU, BAHAMAS",NaN,BHS,Bahamas
9,24000010,Bahamas Leaks,"GROUND FLOOR, GOODMAN'S BAY CORPORATE CE, P.O. BOX N 3933, NASSAU, BAHAMAS",NaN,BHS,Bahamas


In [14]:
bahamas_address_raw.tail(25)

,node_id,sourceID,address,name,country_codes,countries
526,24000527,Bahamas Leaks,"SUITE 2, GROSVENOR CLOSE, SHIRLEY STREET & GROSVENOR CLOSE P.O. BOX CB-11287, NASSAU, BAHAMAS",NaN,BHS,Bahamas
527,24000528,Bahamas Leaks,"INTERNATIONAL HOUSE #1 VIRGINIA STREET, P.O. BOX N-4913, NASSAU, BAHAMAS",NaN,BHS,Bahamas
528,28000001,Bahamas Leaks,"Van Bronckhoestlaan, 20 Wassenaar, 2242 PZ, Netherlands, The",NaN,NLD,Netherlands
529,28000002,Bahamas Leaks,"Alameda Urquijo, N°8, 1 Izq. 48008 Bilbao, Bizkaia, Spain",NaN,ESP,Spain
530,28000003,Bahamas Leaks,"Linhartova 19, 61000 Ljubljana, Slovenia",NaN,SVN,Slovenia
531,28000004,Bahamas Leaks,"Calle Mahonia, no. 2, Edificio Portico, Planta 5th, Avenida de los Andes, Campo de las Naciones, Madrid, Spain",NaN,ESP,Spain
532,28000005,Bahamas Leaks,"Manzana de Obarrio, 20- Obarrio, Panamá, Rep de Panamá",NaN,PAN,Panama
533,28000006,Bahamas Leaks,"TAMARIT, 134, BAJOS 08015, Barcelona España",NaN,ESP,Spain
534,28000007,Bahamas Leaks,"Building #33, Akram Zeiter Street, Abdoun, Amman, 11814, P.O. Box 140825, Jordan",NaN,JOR,Jordan
535,28000008,Bahamas Leaks,"Estacion Maritima, S/N, Denia-03700, Spain",NaN,ESP,Spain


In [15]:
bahamas_address_raw['countries'].value_counts()

Bahamas           530
Spain               9
Greece              3
Panama              2
Venezuela           1
United States       1
United Kingdom      1
Slovenia            1
Netherlands         1
Jordan              1
Hungary             1
Name: countries, dtype: int64

In [16]:
bahamas_address_raw[bahamas_address_raw['countries'] == 'Spain']

,node_id,sourceID,address,name,country_codes,countries
529,28000002,Bahamas Leaks,"Alameda Urquijo, N°8, 1 Izq. 48008 Bilbao, Bizkaia, Spain",NaN,ESP,Spain
531,28000004,Bahamas Leaks,"Calle Mahonia, no. 2, Edificio Portico, Planta 5th, Avenida de los Andes, Campo de las Naciones, Madrid, Spain",NaN,ESP,Spain
533,28000006,Bahamas Leaks,"TAMARIT, 134, BAJOS 08015, Barcelona España",NaN,ESP,Spain
535,28000008,Bahamas Leaks,"Estacion Maritima, S/N, Denia-03700, Spain",NaN,ESP,Spain
539,28000012,Bahamas Leaks,"San Prudencio 29, 20808 Getaria, Guipuzcoa, Spain",NaN,ESP,Spain
541,28000014,Bahamas Leaks,"Corsega 337, 08037, Barcelona, Spain",NaN,ESP,Spain
547,28000020,Bahamas Leaks,"Calle San Nicolas, 9 - 2 dcha, 28013, Madrid, Spain",NaN,ESP,Spain
549,28000022,Bahamas Leaks,"185 - 187 Airbau, 08021 Barcelona, Spain",NaN,ESP,Spain
550,28000023,Bahamas Leaks,"Calle General Pardinas 92-1, 28006, Madrid, Spain",NaN,ESP,Spain


In [10]:
offshore_address_raw.head(25)

# Some have punctuation, some don't.
# Might want to try looking at it by country.
# Change it to title case

,node_id,sourceID,address,name,country_codes,countries
0,2004267,Offshore Leaks,One Bearer Secured Debenture,NaN,XXX,Not identified
1,67277,Offshore Leaks,"11 Coomber Road, The Peak, Hong Kong",NaN,HKG,Hong Kong
2,67346,Offshore Leaks,"4 Irish Place 2nd Floor, Gibraltar.",NaN,GIB,Gibraltar
3,67278,Offshore Leaks,"11A, Branksome 3 Tregunter Path Mid-Levels Hong Kong",NaN,HKG,Hong Kong
4,67282,Offshore Leaks,14 Lysander Road West Mailing Kent ME19 4TT,NaN,GBR,United Kingdom
5,67293,Offshore Leaks,"15 Park Rd,Sorrento, Vic 3943, Australia.",NaN,AUS,Australia
6,67294,Offshore Leaks,16-1/F NO. 125 SUNG THE ROAD TAIPEI TAIWAN,NaN,TWN,Taiwan
7,67297,Offshore Leaks,186A Holland Road Toho Mansion Singapore 278583,NaN,SGP,Singapore
8,67298,Offshore Leaks,"2 Kastoros Street, CY-1087 Nicosia, Cyprus.",NaN,CYP,Cyprus
9,67299,Offshore Leaks,200 Sheridan Ave Suite 103 Palo Alto CA,NaN,USA,United States


In [11]:
panama_address_raw.head(25)

# Includes special characters like `/t` and `"`
# Might be able to split some of it on `;`, but not all are formatted that way. Also the zip and country are flipped on some
# Standardize to title case
# Some include the country, some don't

,node_id,sourceID,address,name,country_codes,countries
0,14000001,Panama Papers,-\t27 ROSEWOOD DRIVE #16-19 SINGAPORE 737920,NaN,SGP,Singapore
1,14000002,Panama Papers,"""Almaly Village"" v.5, Almaty Kazakhstan",NaN,KAZ,Kazakhstan
2,14000003,Panama Papers,"""Cantonia"" South Road St Georges Hill Weybridge, Surrey",NaN,GBR,United Kingdom
3,14000004,Panama Papers,"""CAY-OS"" NEW ROAD; ST.SAMPSON; GUERNSEY; CHANNEL ISLANDS",NaN,GGY,Guernsey
4,14000005,Panama Papers,"""Chirag"" Plot No 652; Mwamba Road; Kizingo; Mombasa; 80100; Kenya",NaN,KEN,Kenya
5,14000006,Panama Papers,"""La Diana""; 16 rue Del Respiro; MC 98000 Monaco.",NaN,MCO,Monaco
6,14000007,Panama Papers,"""Les hauts de St Jean""; 17 avenue Perdtemts; Canton de Vaud; CH-1260 Nyon (Suisse)",NaN,CHE,Switzerland
7,14000008,Panama Papers,"""Les Tattes""; Bursinel; Vaud; Switzerland",NaN,CHE,Switzerland
8,14000009,Panama Papers,"""Quai Kennedy"" 1 boulevard Louis II Monaco MC98000",NaN,MCO,Monaco
9,14000010,Panama Papers,"""Sofia House"" 48 Church Street; 1st Floor; Hamilton HM 12; Bermuda",NaN,BMU,Bermuda


In [12]:
paradise_address_raw.head(25)

# Looks like this is the one with the most standardized punctuation. Should be able to split on ';'
# However, country formats are different. Taiwan is sometimes 'Taiwan', sometimes 'People's Republic of China'. Will need to standardize that
# 'address' column only includes the first part of the address. 'name' includes the full address
# The 'name' column sometimes truncates the first line of the address. This seems to be based on a ';' split

,node_id,sourceID,address,name,country_codes,countries
0,81014050,Paradise Papers - Appleby,6B Chenyu Court,6B Chenyu Court; 22-24 Kennedy Road; Hong Kong,HKG,Hong Kong
1,81014052,Paradise Papers - Appleby,15C Suchun Industrial Square,15C Suchun Industrial Square; Suzhou Industrial Park; 215126 Suzhou; People's Republic of China,CHN,China
2,81014055,Paradise Papers - Appleby,"8F., No. 68","8F., No. 68; Minfu 13th St. Taoyuan County; 330; Taiwan",TWN,Taiwan
3,81014056,Paradise Papers - Appleby,"No. 66, Ln. 20","No. 66, Ln. 20; Dafu Rd., Shengang Township; 429 Taichung County; Taiwan",TWN,Taiwan
4,81014057,Paradise Papers - Appleby,"1F., No. 18, Ln. 274","1F., No. 18, Ln. 274; Chang'an St., Luzhou City; Taipei County 247, Taiwan; People's Republic of China",CHN,China
5,81014058,Paradise Papers - Appleby,8 Middle Gap Road,"8 Middle Gap Road; The Peak, Hong Kong; Hong Kong",HKG,Hong Kong
6,81014060,Paradise Papers - Appleby,8 Sing Woo Crescent,8 Sing Woo Crescent; Happy Valley; Hong Kong; People's Republic of China,CHN,China
7,81014062,Paradise Papers - Appleby,Kings Court,"Kings Court; 1st Floor, Bay Street; PO Box N-3944; Nassau; Bahamas",BHS,Bahamas
8,81014067,Paradise Papers - Appleby,No. 60,No. 60; Baihuazhou; Suzhou City; Jiangsu Province; People's Republic of China,CHN,China
9,81014068,Paradise Papers - Appleby,Kanjata Rd.,Kanjata Rd.; Lavington; Nairobi; Kenya,KEN,Kenya


In [49]:
paradise_address_raw.rename(columns={'address': 'line1', 'name':'address'}, inplace=True)
paradise_address_raw.rename(columns={'line1':'name'}, inplace=True)
paradise_address_raw = paradise_address_raw[keep_cols]
paradise_address_raw.head()


,node_id,sourceID,address,name,country_codes,countries
0,81014050,Paradise Papers - Appleby,6B Chenyu Court; 22-24 Kennedy Road; Hong Kong,6B Chenyu Court,HKG,Hong Kong
1,81014052,Paradise Papers - Appleby,15C Suchun Industrial Square; Suzhou Industrial Park; 215126 Suzhou; People's Republic of China,15C Suchun Industrial Square,CHN,China
2,81014055,Paradise Papers - Appleby,"8F., No. 68; Minfu 13th St. Taoyuan County; 330; Taiwan","8F., No. 68",TWN,Taiwan
3,81014056,Paradise Papers - Appleby,"No. 66, Ln. 20; Dafu Rd., Shengang Township; 429 Taichung County; Taiwan","No. 66, Ln. 20",TWN,Taiwan
4,81014057,Paradise Papers - Appleby,"1F., No. 18, Ln. 274; Chang'an St., Luzhou City; Taipei County 247, Taiwan; People's Republic of China","1F., No. 18, Ln. 274",CHN,China


In [50]:
address_df = pd.concat([bahamas_address_raw, offshore_address_raw, panama_address_raw, paradise_address_raw])
address_df.head()

,node_id,sourceID,address,name,country_codes,countries
0,24000001,Bahamas Leaks,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS",NaN,BHS,Bahamas
1,24000002,Bahamas Leaks,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-8188, NASSAU, BAHAMAS",NaN,BHS,Bahamas
2,24000003,Bahamas Leaks,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS",NaN,BHS,Bahamas
3,24000004,Bahamas Leaks,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS",NaN,BHS,Bahamas
4,24000005,Bahamas Leaks,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS",NaN,BHS,Bahamas


In [46]:
address_df.shape

(374955, 6)

In [36]:
print(bahamas_address_raw[bahamas_address_raw['address'].duplicated(keep=False)].sort_values('address').shape)
bahamas_address_raw[bahamas_address_raw['address'].duplicated(keep=False)].sort_values('address')

(2, 6)


,node_id,sourceID,address,name,country_codes,countries
5,24000006,Bahamas Leaks,"303 SHIRLEY STREET, P.O. BOX N-492, NASSAU, BAHAMAS",NaN,BHS,Bahamas
237,24000238,Bahamas Leaks,"303 SHIRLEY STREET, P.O. BOX N-492, NASSAU, BAHAMAS",NaN,BHS,Bahamas


In [53]:
print(offshore_address_raw[offshore_address_raw['address'].str.title().duplicated(keep=False)].sort_values('address').shape)
offshore_address_raw[offshore_address_raw['address'].str.title().duplicated(keep=False)].sort_values('address').head()

(283, 6)


,node_id,sourceID,address,name,country_codes,countries
1352,240446,Offshore Leaks,1 Farrer Drive #02-02 Singapore 259275,NaN,SGP,Singapore
30792,240445,Offshore Leaks,1 Farrer Drive #02-02 Singapore 259275,NaN,SGP,Singapore
16037,178290,Offshore Leaks,1 Taman Serasi #08-07 Singapore 257717,NaN,SGP,Singapore
15517,178289,Offshore Leaks,1 Taman Serasi #08-07 Singapore 257717,NaN,SGP,Singapore
10440,240773,Offshore Leaks,10 Cuscaden Walk #13-04 Singapore 249693,NaN,SGP,Singapore


In [39]:
print(panama_address_raw[panama_address_raw['address'].str.title().duplicated(keep=False)].sort_values('address').shape)
panama_address_raw[panama_address_raw['address'].str.title().duplicated(keep=False)].sort_values('address')

(10, 6)


,node_id,sourceID,address,name,country_codes,countries
20807,14021331,Panama Papers,6 SALAH EL DIN STREET; ZAMALEK; CAIRO; EGYPT,NaN,EGY,Egypt
20808,14021332,Panama Papers,6 salah El Din Street; Zamalek; Cairo; Egypt,NaN,EGY,Egypt
27535,14028059,Panama Papers,ARANGO ORILLAC BLDG.; 2ND FLOOR; 54TH STREET; PANAMA,NaN,PAN,Panama
27534,14028058,Panama Papers,Arango Orillac Bldg.; 2nd Floor; 54th Street; Panama,NaN,PAN,Panama
65080,14065605,Panama Papers,Rincon 531; Piso 7; Oficina 702; Uruguay,NaN,URY,Uruguay
65087,14065612,Panama Papers,Rincon 531; Piso 7; Oficina 702; Uruguay,NaN,URY,Uruguay
77138,14077663,Panama Papers,SECOND FLOOR; FORUM 4; GRENVILLE STREET; ST HELIER; JERSEY; JE2 4UF,NaN,JEY,Jersey
78528,14079053,Panama Papers,SUITE 13; FIRST FLOOR; OLIAJI TRADE CENTRE; FRANCIS RACHEL STREET; VICTORIA; MAHE; SEYCHELLES,NaN,SYC,Seychelles
77142,14077667,Panama Papers,Second Floor; Forum 4; Grenville Street; St Helier; Jersey; JE2 4UF,NaN,JEY,Jersey
78558,14079083,Panama Papers,Suite 13; First Floor; Oliaji Trade Centre; Francis Rachel Street; Victoria; Mahe; Seychelles,NaN,SYC,Seychelles


In [51]:
print(paradise_address_raw[paradise_address_raw['address'].str.title().duplicated(keep=False)].sort_values('address').shape)
paradise_address_raw[paradise_address_raw['address'].str.title().duplicated(keep=False)].sort_values('address').head()

(1029, 6)


,node_id,sourceID,address,name,country_codes,countries
6629,81026137,Paradise Papers - Appleby,"#05-06, Pan Pacific Serviced Suites; 96 Somerset Road; Singapore 238163","#05-06, Pan Pacific Serviced Suites",SGP,Singapore
14915,81036861,Paradise Papers - Appleby,"#05-06, Pan Pacific Serviced Suites; 96 Somerset Road; Singapore 238163","#05-06, Pan Pacific Serviced Suites",SGP,Singapore
51186,81084819,Paradise Papers - Appleby,"#19-01 Ardmore II, Ardmore Park; 259947; Singapore","#19-01 Ardmore II, Ardmore Park",SGP,Singapore
51185,81084818,Paradise Papers - Appleby,"#19-01 Ardmore II, Ardmore Park; 259947; Singapore","#19-01 Ardmore II, Ardmore Park",SGP,Singapore
53975,81002313,Paradise Papers - Appleby,1 Castle Street; Castletown; Isle of Man IM9 1LF,1 Castle Street,IMN,Isle of Man


In [52]:
print(address_df[address_df['address'].str.title().duplicated(keep=False)].sort_values('address').shape)
address_df[address_df['address'].str.title().duplicated(keep=False)].sort_values('address').head()

(1668, 6)


,node_id,sourceID,address,name,country_codes,countries
14915,81036861,Paradise Papers - Appleby,"#05-06, Pan Pacific Serviced Suites; 96 Somerset Road; Singapore 238163","#05-06, Pan Pacific Serviced Suites",SGP,Singapore
6629,81026137,Paradise Papers - Appleby,"#05-06, Pan Pacific Serviced Suites; 96 Somerset Road; Singapore 238163","#05-06, Pan Pacific Serviced Suites",SGP,Singapore
51186,81084819,Paradise Papers - Appleby,"#19-01 Ardmore II, Ardmore Park; 259947; Singapore","#19-01 Ardmore II, Ardmore Park",SGP,Singapore
51185,81084818,Paradise Papers - Appleby,"#19-01 Ardmore II, Ardmore Park; 259947; Singapore","#19-01 Ardmore II, Ardmore Park",SGP,Singapore
77161,33000032,Paradise Papers - Bahamas corporate registry,"#4 EVERGREEN BUSINESS CENTRE CORAL ROAD, FREEPORT, GRAND BAHAMA",NaN,BHS,Bahamas


In [54]:
address_df['countries'].value_counts()

China                                31984
Hong Kong                            27043
United States                        23819
Barbados                             19334
Aruba                                17510
Taiwan                               15326
United Kingdom                       10729
Singapore                             7065
Russia                                5963
British Virgin Islands                5221
Switzerland                           4721
Bermuda                               3913
Jersey                                3909
Indonesia                             3395
Malaysia                              3196
Panama                                3001
Canada                                2980
Cyprus                                2628
United Arab Emirates                  2611
Australia                             2403
Isle of Man                           2368
Cayman Islands                        2273
Bahamas                               2164
Guernsey   

In [62]:
address_df['address'].str.lower().str.split(' ')[:5]

# look at ones where the last word when split on ' ' is the same as the 'countries' column

0                                            [annex, frederick, &, shirley, sts,, p.o., box, n-4805,, nassau,, bahamas]
1                                            [suite, e-2,union, court, building,, p.o., box, n-8188,, nassau,, bahamas]
2                                             [lyford, cay, house,, lyford, cay,, p.o., box, n-7785,, nassau,, bahamas]
3    [p.o., box, n-3708, bahamas, financial, centre,, p.o., box, n-3708, shirley, &, charlotte, sts,, nassau,, bahamas]
4                                [lyford, cay, house,, 3rd, floor,, lyford, cay,, p.o., box, n-3024,, nassau,, bahamas]
Name: address, dtype: object

In [64]:
address_df['address'].str.lower().str.split(' ')[-5:]

# look for rows where the end is numbers (ie probably zip/postal codes)

223345                                     [manase,, samoa]
223346              [vailele-uta,, faleata, sisifo,, samoa]
223347             [lotopa,, falelatai, &, samatau,, samoa]
223348                     [sogi,, vaisigano, no.2,, samoa]
223349    [viviena.schaaf,, `,, faasaleleaga, no.3,, samoa]
Name: address, dtype: object